In [1]:
"""Machine learning functions"""

import logging
import json
import joblib
import random
import pandas as pd
from pydantic import BaseModel, Field, validator
from sklearn.neighbors import NearestNeighbors



In [2]:

class User(BaseModel):
    """Use this data model to parse the request body JSON."""
    user_input: str = Field(..., example=
        """I am having trouble sleeping, I'm stressed and I don't want to eat. I would like to have an appetite, sleep well, and be able to relax.""")


In [3]:
a_user = User(user_input='I want to have a appetite and be able to sleep. I do not like being tired, cranky and having no energy.')

In [4]:
nn = joblib.load(r'C:\Users\whats\DS_Unit_3\Build\best-buds\app\jl_knn.joblib')
vectorizer = joblib.load(r'C:\Users\whats\DS_Unit_3\Build\best-buds\app\jl_tfidf.joblib')

MJ = pd.read_csv('https://raw.githubusercontent.com/tt-dspt-200/best-buds/main/data/MJ.csv')

model_food = vectorizer.transform([a_user.user_input])

In [5]:
strain_name_list = []
description_list = []
strain_effects_list = []
strain_ailments_list = []

for i in range(4):
    strain_name = (MJ.loc[nn.kneighbors(model_food.todense())[1][0][i]]['Strain']),
    description = (MJ.loc[nn.kneighbors(model_food.todense())[1][0][i]]['Description']),
    strain_effects = (MJ.loc[nn.kneighbors(model_food.todense())[1][0][i]]['Effects_y']),
    strain_ailments = (MJ.loc[nn.kneighbors(model_food.todense())[1][0][i]]['Ailment'])
    strain_name_list.append(strain_name),
    description_list.append(description),
    strain_effects_list.append(strain_effects),
    strain_ailments_list.append(strain_ailments)



In [6]:
strain_df = pd.DataFrame([strain_name_list, description_list, strain_effects_list, strain_ailments_list], index = ['name', 'description', 'effects', 'ailments'],).T
# strain_df = pd.DataFrame([strain_name_list, strain_ailments_list], index = ['name',  'ailments']).T
strain_df


,name,description,effects,ailments
0,"(EAST COAST SOUR DIESEL,)",(East Coast Sour Diesel is a fast-acting attit...,"(Happy, Focused, Energetic, Uplifted, Euphoric...","Muscle Spasms, Stress, Pain, Depression, Infla..."
1,"(NIGHT NURSE,)",(Night Nurse is an indica-dominant strain from...,"(Relaxed, Sleepy, Happy, Tingly, Hungry, Dry M...","Pain, Insomnia, Stress, Muscle Spasms"
2,"(OGRE BERRY,)",(Ogre Berry is an indica-dominant strain culti...,"(Focused, Relaxed, Happy, Sleepy, Euphoric, Dr...","Stress, Lack of Appetite, Insomnia, Pain, Depr..."
3,"(DJANGO,)","(Django means “I awake,” in Romani, and this u...","(Energetic, Uplifted, Happy, Talkative, Euphor...","Stress, Depression, Insomnia, Muscle Spasms"


In [7]:
strain_df.dtypes

name           object
description    object
effects        object
ailments       object
dtype: object

In [8]:
df =strain_df.convert_dtypes()
df.dtypes

name           object
description    object
effects        object
ailments       string
dtype: object

In [9]:
strain_json = strain_df.to_json(orient="records")
strain_json


'[{"name":["EAST COAST SOUR DIESEL"],"description":["East Coast Sour Diesel is a fast-acting attitude lifter that patients love for its truly sativa effects. This strain has a sour lemon smell and taste that\\u2019s crisp and uplifting. East Coast Sour Diesel is powerful so new patients should be cautious. Those looking to alleviate anxiety or get some sleep may want to skip this high energy medicine."],"effects":["Happy, Focused, Energetic, Uplifted, Euphoric, Dry Mouth, Paranoid, Anxious"],"ailments":"Muscle Spasms, Stress, Pain, Depression, Inflammation"},{"name":["NIGHT NURSE"],"description":["Night Nurse is an indica-dominant strain from Reeferman Seeds that combines genetics from BC Hash Plant, Harmony, and Fire OG. It\\u2019s commonly used to fight anxiety, stress, insomnia and chronic pain. Are you having trouble sleeping? This 80\\/20 indica provides a polite balance of relaxing cerebral and body effects, making this the perfect nighttime sleeping aid."],"effects":["Relaxed, S

In [10]:
parsed = json.loads(strain_json)
parsed


[{'name': ['EAST COAST SOUR DIESEL'],
  'description': ['East Coast Sour Diesel is a fast-acting attitude lifter that patients love for its truly sativa effects. This strain has a sour lemon smell and taste that’s crisp and uplifting. East Coast Sour Diesel is powerful so new patients should be cautious. Those looking to alleviate anxiety or get some sleep may want to skip this high energy medicine.'],
  'effects': ['Happy, Focused, Energetic, Uplifted, Euphoric, Dry Mouth, Paranoid, Anxious'],
  'ailments': 'Muscle Spasms, Stress, Pain, Depression, Inflammation'},
 {'name': ['NIGHT NURSE'],
  'description': ['Night Nurse is an indica-dominant strain from Reeferman Seeds that combines genetics from BC Hash Plant, Harmony, and Fire OG. It’s commonly used to fight anxiety, stress, insomnia and chronic pain. Are you having trouble sleeping? This 80/20 indica provides a polite balance of relaxing cerebral and body effects, making this the perfect nighttime sleeping aid.'],
  'effects': ['R

In [15]:
result = json.dumps(parsed)
result

'[{"name": ["EAST COAST SOUR DIESEL"], "description": ["East Coast Sour Diesel is a fast-acting attitude lifter that patients love for its truly sativa effects. This strain has a sour lemon smell and taste that\\u2019s crisp and uplifting. East Coast Sour Diesel is powerful so new patients should be cautious. Those looking to alleviate anxiety or get some sleep may want to skip this high energy medicine."], "effects": ["Happy, Focused, Energetic, Uplifted, Euphoric, Dry Mouth, Paranoid, Anxious"], "ailments": "Muscle Spasms, Stress, Pain, Depression, Inflammation"}, {"name": ["NIGHT NURSE"], "description": ["Night Nurse is an indica-dominant strain from Reeferman Seeds that combines genetics from BC Hash Plant, Harmony, and Fire OG. It\\u2019s commonly used to fight anxiety, stress, insomnia and chronic pain. Are you having trouble sleeping? This 80/20 indica provides a polite balance of relaxing cerebral and body effects, making this the perfect nighttime sleeping aid."], "effects": [